In [ ]:
!pip install -q transformers einops accelerate langchain bitsandbytes
!pip install -qqq openai
!pip install -Uqqq chromadb


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 44.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.5/287.5 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.2/104.2 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.3/268.3 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install -q tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 13.5 MB/s eta 0:00:00


In [ ]:
import os
import transformers
from transformers import AutoTokenizer
from langchain import HuggingFacePipeline
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import langchain
import chromadb
import openai
import torch

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
#Set up HuggingFace Pipeline with Llama-2-7b-chat-hf model
model = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
      "text-generation", #task
      model=model,
      tokenizer=tokenizer,
      torch_dtype=torch.bfloat16,
      trust_remote_code=True,
      device_map="auto",
      max_length=1000,
      do_sample=True,
      top_k=10,
      num_return_sequences=1,
      eos_token_id=tokenizer.eos_token_id
)

#LLM intialized in HuggingFace Pipeline wrapper
llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature':0})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:
!pip install -q jq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.6/657.6 kB 9.3 MB/s eta 0:00:00


In [ ]:
from langchain.document_loaders import JSONLoader
loader=JSONLoader(file_path="hotle.json",jq_schema=".",json_lines=True,text_content=False)
docs=loader.load()
for doc in docs:
    print(doc.page_content)

{"id": 1, "name": "Hotel Plaza", "location": "13 Street B1 Sector, Gokulpuri, Delhi", "cuisine": "Cake, Coldrink, Energy drink, snacks"}
{"id": 2, "name": "Cafe Royale", "location": "22 Park Avenue, Sector 5, Mumbai", "cuisine": "Coffee, Sandwiches, Pastries, Burgers"}
{"id": 3, "name": "The Green Leaf", "location": "8A/1 Park Street, Kolkata", "cuisine": "Vegetarian, Indian, Chinese, Continental"}
{"id": 4, "name": "Seafood Haven", "location": "45 Beach Road, Goa", "cuisine": "Seafood, Grill, Cocktails, Snacks"}
{"id": 5, "name": "Pizzeria Italia", "location": "10 Piazza Navona, Rome", "cuisine": "Pizza, Pasta, Italian, Gelato"}


In [ ]:
os.environ["OPENAI_API_KEY"] = ""

In [ ]:
# Initialize the open-source embedding function, default: text-embedding-ada-002
embedding_function = OpenAIEmbeddings()

In [ ]:
# Load it into ChromaDB
db = Chroma.from_documents(docs, embedding_function)

In [ ]:
db.similarity_search('hotel in rome', k=1)

[Document(page_content='{"id": 5, "name": "Pizzeria Italia", "location": "10 Piazza Navona, Rome", "cuisine": "Pizza, Pasta, Italian, Gelato"}', metadata={'seq_num': 5, 'source': '/content/hotle.json'})]

In [ ]:
template = """
You are a assiatance of a food delivery company where you have to guide the user on the bases of the question that are been aksed, you do not respond as 'User' or pretend to be 'User'. You only respond once as 'Assistant'.

Question - {context}
Answer
"""


In [ ]:

#Intiliaze prompt using prompt template via langchain
prompt = PromptTemplate(template=template, input_variables=["context"])

In [ ]:
print(
    prompt.format(
        context = "what are the best cake available in Lucknow"
    )
)


You are a assiatance of a food delivery company where you have to guide the user on the bases of the question that are been aksed, you do not respond as 'User' or pretend to be 'User'. You only respond once as 'Assistant'.

Question - what are the best cake available in Lucknow
Answer



In [ ]:
#Chain to have all components together and query the LLM
chain_type_kwargs = {"prompt": prompt}

In [ ]:

chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 1}),
    chain_type_kwargs=chain_type_kwargs,
)

In [ ]:
def chain_response(context):
  resp = chain.run(context)
  return resp

def get_response(context):
  qa_prompt = prompt.format(
        context = context
    )
  resp = llm(qa_prompt)
  return resp

In [ ]:
resp = get_response("can you deliver a cake to mee")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [ ]:
resp2 = chain_response('i want a vegetarian food is it avaliable in any hotel, where can i find that')

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [ ]:
# Formatted printing
def print_response(response: str):
    text = "\n".join(textwrap.wrap(response, width=80))
    return text.split('Answer ')[1].replace("\n", " ")

In [ ]:
print_response(resp2)

'You have reached Hotel Plaza, located at 13 Street B1 Sector, Gokulpuri, Delhi. We offer a wide range of delicious food items including cakes, cold drinks, energy drinks, and snacks. What can I get for you today?'

In [ ]:
print_response(resp)

"Of course, I'd be happy to help! Can you please provide me with more details about the cake you would like to order? What is the occasion, what is the flavor, and how many people are you expecting?"